# **Predictive Default Risk Assessor V.01**

# TODO

* Base model 
* Comparison
* Specialised
* For small entities - Examples?
* Backtest - All sectors 
* Understanding the model across all sectors/industries
* Any markets - consumer goods, industries
* UI last step after backtesting

In [1]:
model_inputs = {
    "profitability": {
        "class_weight": 0.30,
        "weights": [1.0], 
        "metrics": {
            "oper_margin": {
                "lower_is_better": False,
                "thresholds": [
                    (40, float("inf")),
                    (35, 39),
                    (30, 34),
                    (25, 29),
                    (20, 24),
                    (15, 19),
                    (10, 14),
                    (5, 9),
                    (float("-inf"), 0)
                ],
            }
        },
    },
    "leverage_coverage": {
        "class_weight": 0.55,
        "weights": [0.4, 0.3, 0.3],
        "metrics": {
            "tot_debt_to_tot_eqy": {
                "lower_is_better": True,
                "thresholds": [
                     (float("-inf"), 2.0),
                     (2.0, 16.0),
                     (16.0, 24.0),
                     (24.0, 33.0),
                     (33.0, 43.0),
                     (43.0, 54.0),
                     (54.0, 68.0),
                     (68.0, 94.0),
                     (94.0, float("inf")),
                ],
            },
            "tot_debt_to_ebitda": {
                "lower_is_better": True,
                "thresholds": [
                    (float("-inf"), 0.09),
                    (0.09, 0.49),
                    (0.49, 0.9),
                    (0.9, 1.36),
                    (1.36, 1.68),
                    (1.68, 2.26),
                    (2.26, 3.27),
                    (3.27, 4.4),
                    (4.4, float("inf")),
                ],
            },
            "ebitda_to_tot_int_exp": {
                "lower_is_better": False,
                "thresholds": [
                    (25, float("inf")),
                    (20, 25),
                    (15, 20),
                    (10, 15),
                    (5, 10),
                    (3, 5),
                    (1, 3),
                    (0, 1),
                    (float("-inf"), 0),
                ],
            },
        },
    },
    "efficiency": {
        "class_weight": 0.15,
        "weights": [0.5, 0.5],
        "metrics": {
            "return_on_asset": {
                "lower_is_better": False,
                "thresholds": [
                    (0.15, float("inf")),
                    (0.10, 0.15),
                    (0.08, 0.10),
                    (0.06, 0.08),
                    (0.04, 0.06),
                    (0.02, 0.04),
                    (0.00, 0.02),
                    (-0.02, 0.00),
                    (float("-inf"), -0.02)
                ],
            },
            "asset_turnover": {
                "lower_is_better": False,
                "thresholds": [
                    (4.0, float("inf")),
                    (3.0, 4.0),
                    (2.0, 3.0),
                    (1.5, 2.0),
                    (1.0, 1.5),
                    (0.75, 1.0),
                    (0.5, 0.75),
                    (0.25, 0.5),
                    (float("-inf"), 0.25)
                ],
            },
        },
    },
}

In [2]:
import pandas as pd
import numpy as np

In [3]:
class CreditRatingCalculator:
    def __init__(self, metrics):
        self.metrics = metrics
        
    def _calculate_metric_score(self, metric, thresholds, inverse):
        for score, (lower, upper) in enumerate(thresholds, start=1):
            if (inverse and metric <= upper) or (not inverse and metric >= lower):
                return score
        return len(thresholds) // 2 # else return the middle score

    def _calculate_category_score(self, category_metrics, ratios):
        total_weighted_score = 0

        for metric, weight in zip(
            category_metrics["metrics"].items(), category_metrics["weights"]
        ):
            metric_name, metric_data = metric
            value = ratios[metric_name]
            score = self._calculate_metric_score(
                value, metric_data["thresholds"], metric_data["lower_is_better"]
            )
            total_weighted_score += score * weight

        return total_weighted_score

    def _calculate_scores(self, ratios):
        scores = {}
        for category, category_data in self.metrics.items():
            category_score = self._calculate_category_score(category_data, ratios)
            scores[category] = category_score
        return scores

    def _calculate_weighted_score(self, scores):
        weights = {
            category: category_data["class_weight"]
            for category, category_data in self.metrics.items()
        }
        return sum(scores[category] * weight for category, weight in weights.items())

    def _determine_credit_rating(self, weighted_score):
        # credit_ratings = [
        #     (1.5, "Aaa"),
        #     (2.5, "Aa"),
        #     (3.5, "A"),
        #     (4.5, "Baa"),
        #     (5.5, "Ba"),
        #     (6.5, "B"),
        #     (7.5, "Caa"),
        #     (8.5, "Ca"),
        #     (float("inf"), "C"),
        # ]
        
        credit_ratings = [
            (2.5, "Aaa"),
            (3.5, "Aa"),
            (4.5, "A"),
            (5.5, "Baa"),
            (6.5, "Ba"),
            (7.5, "B"),
            (8.5, "Caa"),
            (9.5, "Ca"),
            (float("inf"), "C"),
        ]

        for threshold, rating in credit_ratings:
            if weighted_score < threshold:
                return rating

    def calculate_credit_rating(self, ratios):
        self.scores = self._calculate_scores(ratios)
        self.credit_score = self._calculate_weighted_score(self.scores)
        self.credit_rating = self._determine_credit_rating(self.credit_score)

In [4]:
# df = pd.read_csv("research/JALSH Index_dataset_2000_2024_clean.csv", index_col=0, header=[0, 1])
# classfier = pd.read_excel("research/classification_data.xlsx", index_col=0)
metrics = pd.read_excel("research/metrics_full.xlsx", index_col=0)

In [5]:
company = "TKG SJ Equity"

In [6]:
model_metrics = [
    "oper_margin", 
    "tot_debt_to_tot_eqy", 
    "tot_debt_to_ebitda", 
    "ebitda_to_tot_int_exp", 
    "return_on_asset",
    "asset_turnover",
]

ratios = metrics[company].loc[model_metrics].to_dict()

In [7]:
model = CreditRatingCalculator(model_inputs)
model.calculate_credit_rating(ratios)
print(f"Model Inputs:")
display(ratios)
print("")
print(f"Class Scoring: {model.scores}")
print(f"Credit Score: {model.credit_score}")
print(f"Credit Rating: {model.credit_rating}")

Model Inputs:


{'oper_margin': 12.96540152557296,
 'tot_debt_to_tot_eqy': 56.59317158501908,
 'tot_debt_to_ebitda': 1.430976565821454,
 'ebitda_to_tot_int_exp': 8.796523439549544,
 'return_on_asset': 6.401369750459697,
 'asset_turnover': 0.7242518101367109}


Class Scoring: {'profitability': 7.0, 'leverage_coverage': 5.800000000000001, 'efficiency': 4.0}
Credit Score: 5.890000000000001
Credit Rating: Ba


In [8]:
metrics

,ABG SJ Equity,ADH SJ Equity,AEL SJ Equity,AFE SJ Equity,AFH SJ Equity,AFT SJ Equity,AGL SJ Equity,AIL SJ Equity,AIP SJ Equity,AMS SJ Equity,...,TFG SJ Equity,TGA SJ Equity,THA SJ Equity,TKG SJ Equity,TRU SJ Equity,TSG SJ Equity,VKE SJ Equity,VOD SJ Equity,WBO SJ Equity,WHL SJ Equity
oper_margin,28.202722,14.421976,5.841907,8.061480,19.331788,16.912217,18.427020,87.144999,16.014715,22.805459,...,12.665255,14.666294,4.164911,12.965402,19.765067,27.788494,68.366238,26.210947,4.335948,8.610180
return_on_asset,1.233318,8.247937,7.653897,5.113338,0.099075,12.092518,5.623050,4.356048,11.363074,14.951768,...,10.459391,14.996192,4.690065,6.401370,21.802062,8.697806,5.667759,16.143841,5.572577,9.756574
tot_debt_to_tot_asset,20.212411,19.850619,17.306101,21.207294,0.975794,11.256166,22.294088,0.000000,11.012743,10.662628,...,27.733825,0.800685,17.614912,24.225158,10.837195,34.730336,34.257211,24.905928,2.921138,28.117250
tot_debt_to_tot_cap,64.622361,24.330877,30.528639,31.505856,31.281776,15.862061,29.696387,0.000000,14.466634,15.155702,...,34.369187,2.304921,41.180385,32.873565,13.777841,41.338556,41.120550,37.757389,8.074190,42.591561
tot_debt_to_tot_eqy,242.937113,38.061018,53.463897,47.459330,80.367497,19.268366,43.240614,0.000000,18.092223,20.964517,...,59.437803,2.451614,115.038914,56.593172,21.861382,141.812906,73.289447,65.004940,9.188145,124.097944
asset_turnover,0.108163,1.115640,1.468364,1.175394,0.016101,1.037714,0.482276,0.063254,1.046750,0.798966,...,1.046913,0.992103,0.724942,0.724252,1.210102,0.573161,0.119474,1.076702,2.148323,1.897337
ebitda_margin,NaN,18.949128,8.609112,11.232598,27.476468,21.956225,26.502615,NaN,18.699604,29.452630,...,17.680716,20.215690,9.506585,27.292317,23.434060,35.590950,69.210913,35.695467,5.258228,11.653312
tot_debt_to_ebitda,NaN,1.329016,1.722145,1.734404,2.281107,0.607097,2.968238,NaN,0.521484,0.779408,...,1.894263,0.279258,2.297316,1.430977,0.565602,2.552621,3.613919,0.793112,0.311257,1.584613
interest_coverage_ratio,NaN,170.789112,13.542255,5.032983,NaN,13.990275,6.337486,NaN,20.350946,118.117503,...,7.209910,12.612075,7.319247,3.495262,1339.192812,5.561308,3.548919,9.790341,212.325909,12.769602
ebitda_to_tot_int_exp,NaN,219.356343,18.225091,7.064499,7.725572,18.198105,8.820816,NaN,25.139900,128.682471,...,9.788604,14.661929,10.029111,8.796523,1496.857073,7.036121,3.830182,13.355185,278.833027,16.341040
